# **Green Coffee Beans Conference Paper - Convolutional Neural Network**

In [ ]:
# OpenCV 套件，和 Google Drive 無關
import cv2
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')
#img = cv2.imread('gdrive/My Drive')
#from google.colab.patches import cv2_imshow
#cv2_imshow()

Mounted at /content/gdrive


In [ ]:
# Import需要的套件
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
import torchvision.models

#Read image
利用 OpenCV (cv2) 讀入照片並存放在 numpy array 中

In [ ]:
def readfile(path):
    good_path = path + "/good"
    bad_path = path + "/bad"
    good_image_dir = sorted(os.listdir(good_path))
    bad_image_dir = sorted(os.listdir(bad_path))
    x = np.zeros((len(good_image_dir + bad_image_dir), 256, 256, 3), dtype=np.uint8)
    y = np.zeros((len(good_image_dir + bad_image_dir)), dtype=np.uint8)
    for i, file in enumerate(good_image_dir):
        good_img = cv2.imread(os.path.join(good_path, file))
        x[i, :, :] = cv2.resize(good_img,(256, 256))
        y[i] = 1
    for i, file in enumerate(bad_image_dir):
        bad_img = cv2.imread(os.path.join(bad_path, file))
        x[i, :, :] = cv2.resize(bad_img,(256, 256))
        y[i + len(good_image_dir)] = 0
    return x, y

In [ ]:
!ls

gdrive	sample_data


In [ ]:
# 分別將 training set、validation set、testing set 用 readfile 函式讀進來
workspace_dir = 'gdrive/My Drive/dataset/'
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training2"))
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation2"))
print("Size of validation data = {}".format(len(val_x)))
test_x, test_y = readfile(os.path.join(workspace_dir, "testing2"))
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of training data = 3691
Size of validation data = 463
Size of Testing data = 463


# Dataset
在 PyTorch 中，我們可以利用 torch.utils.data 的 Dataset 及 DataLoader 來"包裝" data，使後續的 training 及 testing 更為方便。

Dataset 需要 overload 兩個函數：\_\_len\_\_ 及 \_\_getitem\_\_

\_\_len\_\_ 必須要回傳 dataset 的大小，而 \_\_getitem\_\_ 則定義了當程式利用 [ ] 取值時，dataset 應該要怎麼回傳資料。

實際上我們並不會直接使用到這兩個函數，但是使用 DataLoader 在 enumerate Dataset 時會使用到，沒有實做的話會在程式運行階段出現 error。


In [ ]:
# training 時做 data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), # 隨機將圖片水平翻轉
    transforms.RandomRotation(15), # 隨機旋轉圖片
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
])
# testing 時不需做 data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [ ]:
batch_size = 8 #32以內
train_set = ImgDataset(train_x, train_y, train_transform)
val_set = ImgDataset(val_x, val_y, test_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Model

In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 128, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(128, 128, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(128*8*8, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [ ]:
!pip install thop
import torch
import torchvision
import time
from thop import profile
# 增加可读性
from thop import clever_format

# 可替换为自己的模型及输入
start = time.time()
model = torchvision.models.vgg16(pretrained=False, progress=True, num_classes = 2)
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ))
flops, params = clever_format([flops, params], "%.3f")
end = time.time()
print(flops, params)
print(end-start)

# 可替换为自己的模型及输入
start = time.time()
model = torchvision.models.resnet50(pretrained=False, progress=True, num_classes = 2)
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ))
flops, params = clever_format([flops, params], "%.3f")
end = time.time()
print(flops, params)
print(end-start)

# 可替换为自己的模型及输入
start = time.time()
model = torchvision.models.densenet121(pretrained=False, progress=True, num_classes = 2)
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ))
flops, params = clever_format([flops, params], "%.3f")
end = time.time()
print(flops, params)
print(end-start)

# 可替换为自己的模型及输入
start = time.time()
model = torchvision.models.mobilenet_v2(pretrained=False, progress=True, num_classes = 2)
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ))
flops, params = clever_format([flops, params], "%.3f")
end = time.time()
print(flops, params)
print(end-start)



[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[WARN] Cannot find rule for <class 'torchvision.models.vgg.VGG'>. Treat it as zero Macs and zero Params.
15.480G 134.269M
2.540384531021118
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class '

In [ ]:
vgg16 = torchvision.models.vgg16(pretrained=False, progress=True, num_classes = 2)

In [ ]:
resNet18 = torchvision.models.resnet18(pretrained=False, progress=True, num_classes = 2)

In [ ]:
resNet50 = torchvision.models.resnet50(pretrained=False, progress=True, num_classes = 2)

In [ ]:
denseNet121 = torchvision.models.densenet121(pretrained=False, progress=True, num_classes = 2)

In [ ]:
mobilenet = torchvision.models.mobilenet_v2(pretrained=False, progress=True, num_classes = 2)

In [ ]:
!pip install pretrainedmodels
import pretrainedmodels
model_name = 'xception'
xception = pretrainedmodels.__dict__[model_name](num_classes = 2, pretrained = False)

In [ ]:
!pip install thop
import torch
import torchvision
import time
from thop import profile
# 增加可读性
from thop import clever_format
# 可替换为自己的模型及输入
start = time.time()
model = xception
input = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input, ))
flops, params = clever_format([flops, params], "%.3f")
end = time.time()
print(flops, params)
print(end-start)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'pretrainedmodels.models.xception.SeparableConv2d'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'pretrainedmodels.models.xception.Block'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'pretrainedmodels.models.xception.Xception'>. Treat it as zero Macs and zero Params.
4.574G 20.811M
0.24268007278442383


In [ ]:
!apt install psmisc

# Training

使用 training set 訓練，並使用 validation set 尋找好的參數

In [ ]:
#model = Classifier().cuda()
model = vgg16.cuda()
#model = resNet18.cuda()
#model = resNet101.cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # optimizer 使用 Adam
num_epoch = 50

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_loader):
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(data[0].cuda()) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
        batch_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()
            '''
            if((epoch+1) % 10 == 0):
              torch.save(model.state_dict(), 'gdrive/My Drive/dataset/teacher_model_resNet101.bin')
            '''

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))

[001/100] 140.57 sec(s) Train Acc: 0.926307 Loss: 0.032252 | Val Acc: 0.928726 loss: 0.026785
[002/100] 141.25 sec(s) Train Acc: 0.929829 Loss: 0.029289 | Val Acc: 0.928726 loss: 0.027186


得到好的參數後，我們使用 training set 和 validation set 共同訓練（資料量變多，模型效果較好）

In [ ]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)
train_val_set = ImgDataset(train_val_x, train_val_y, train_transform)
train_val_loader = DataLoader(train_val_set, batch_size=batch_size, shuffle=True)

In [ ]:
#model_best = Classifier().cuda()
#model_best = vgg16.cuda()
model_best = resNet18.cuda()
#model_best = resNet50.cuda()
#model_best = denseNet121.cuda()
#model_best = mobilenet.cuda()
#model_best = xception.cuda()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.0001) # optimizer 使用 Adam
num_epoch = 200

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
        if((epoch+1) % 10 == 0):
          torch.save(model_best.state_dict(), 'gdrive/My Drive/dataset/teacher_model_resNet18.bin')

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

[001/200] 70.07 sec(s) Train Acc: 0.941502 Loss: 0.018232
[002/200] 70.41 sec(s) Train Acc: 0.935243 Loss: 0.019876
[003/200] 69.66 sec(s) Train Acc: 0.941743 Loss: 0.018213
[004/200] 69.74 sec(s) Train Acc: 0.943909 Loss: 0.018192
[005/200] 69.79 sec(s) Train Acc: 0.946076 Loss: 0.016961
[006/200] 69.65 sec(s) Train Acc: 0.946798 Loss: 0.017920
[007/200] 69.66 sec(s) Train Acc: 0.949206 Loss: 0.016227
[008/200] 69.78 sec(s) Train Acc: 0.948965 Loss: 0.016555
[009/200] 69.70 sec(s) Train Acc: 0.951131 Loss: 0.015590
[010/200] 570.08 sec(s) Train Acc: 0.950891 Loss: 0.015499
[011/200] 69.93 sec(s) Train Acc: 0.953779 Loss: 0.014702
[012/200] 69.84 sec(s) Train Acc: 0.954502 Loss: 0.015582
[013/200] 69.94 sec(s) Train Acc: 0.954742 Loss: 0.014700
[014/200] 69.92 sec(s) Train Acc: 0.956428 Loss: 0.013744
[015/200] 69.79 sec(s) Train Acc: 0.957631 Loss: 0.014159
[016/200] 69.83 sec(s) Train Acc: 0.958353 Loss: 0.013423
[017/200] 69.88 sec(s) Train Acc: 0.957872 Loss: 0.013833
[018/200] 69.

# Testing
利用剛剛 train 好的 model 進行 prediction

In [ ]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)

In [ ]:
from sklearn import metrics
target_names=["labels_0"]
for i in range(1,2):
    target_names.append('labels_'+str(i))
print("classification report:")
print(metrics.classification_report(prediction,val_y,target_names=target_names,digits=4))
print("Accuracy: "+str(format(metrics.accuracy_score(prediction,val_y),'0.4f')))
print("Confusion Matrix:")
print (metrics.confusion_matrix(prediction,val_y))

classification report:
              precision    recall  f1-score   support

    labels_0     0.8871    0.9167    0.9016       240
    labels_1     0.9070    0.8744    0.8904       223

    accuracy                         0.8963       463
   macro avg     0.8970    0.8956    0.8960       463
weighted avg     0.8967    0.8963    0.8962       463

Accuracy: 0.8963
Confusion Matrix:
[[220  20]
 [ 28 195]]


In [ ]:
#將結果寫入 csv 檔
with open("gdrive/My Drive/predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))